In [1]:
# numpy == 1.21.5
import numpy as np
# scipy == 1.7.3
import scipy.linalg
# to make nice plots
import matplotlib.pyplot as plt
# optional -- to track how long a simulation will take
from tqdm import tqdm

from steinberg_utils_3vertex import *

# Plot features

In [2]:
# plot formatting
plt.rc("text", usetex=False) # renders LaTeX more quickly
plt.rc("font", family = "serif",size=14) # font specifications
plt.rc("figure",figsize=(14,12)) # figure size
%config InlineBackend.figure_format = 'retina' # retina-display quality

Parameters are defined as $10^x$, where $x$ is uniformly sampled from $(-3, 3)$. The user can specify the endpoints of this range to sample parameter space more broadly, but $-3$ and $3$ are set as the default parameters.

In [3]:
params = random_parameters()

In [4]:
print(params)

[4.01550670e+02 6.34800653e+01 3.00520038e+02 2.44265497e+02
 1.21047019e+01 6.02777829e-03]


The parameters can be determined such that they satisfy the cycle condition on $K$, initializing the graph in an equlibrium steady state.

In [5]:
params = equilibrium_parameters()

In [6]:
print(params)

[9.70944540e+02 7.13723498e-03 6.78107403e-01 2.70267544e+00
 2.72353307e+02 7.97927869e-03]


To generate the figure in the paper, we will manually assign values for the edge labels of the 3-vertex graph. The parameters for the 3-vertex graph are listed in the following order: $[a, b, d, c, f, e]$

In [7]:
#params = [0.08833331, 0.44953403, 0.58597936, 0.02356496, 0.00747019, 0.75898155]

Next, we compute the Laplacian matrix $\mathcal{L}(K)$ for this specific parameterzation of $K$, and the steady state distribution $\pi(K)$ as computed through the Matrix-Tree Theorem.

In [8]:
L = Laplacian_K(params)
print(L)

[[-1.24329785e+03  7.13723498e-03  7.97927869e-03]
 [ 9.70944540e+02 -6.85244638e-01  2.70267544e+00]
 [ 2.72353307e+02  6.78107403e-01 -2.71065472e+00]]


In [9]:
pi = steady_state_MTT_K(params)
print(pi)

[5.87637684e-06 7.99418266e-01 2.00575858e-01]


We can also calculate the cycle affinity $\tilde{A}(C)$ for $K$. The cycle affinity  quantifies the departure from equilibrium that arises from the cycle $C$. We take the absolute value of the cycle affinity in our calculation.

In [10]:
cycle_affinity_K(params)

1.0842021724855044341e-19

# Testing response of a KLD signature to increasing thermodynamic force

Another class of signatures uses the KL-divergence to detect irreversibility. Here, we are going to test whether or not the KL-divergence between the probability of observing a stochastic trajectory and its time-reversed trajectory varies monotonically with increasing thermodynamic force. We will use the mathematical conventions laid out in Martínez et al. 2019.

The entropy production rate $\dot{S}$ can be estimated by the KL-divergence between the probability of observing a stochastic trajectory $\gamma_t$ of length $t$ and the probability to observe the time-reversed trajectory $\tilde{\gamma_t}$.

$$ \dot{S} \geq \dot{S}_{KLD} \equiv \lim_{t \to \infty} \frac{k_B}{t} \mathcal{D} [ P(\gamma_t) || P(\tilde{\gamma_t}) ], $$

where $\mathcal{D}[ p || q ] \equiv \sum_x p(x) \ln{p(x)/q(x)}$ is the KL-divergence between probability distributions $p$ and $q$. We assume that $k_B = 1$.

In Martínez et al 2019, the authors claim that $\dot{S}_{KLD}$ can be expressed as the sum of two entropy productions.

$$\dot{S}_{KLD} = \dot{S}_{aff} + \dot{S}_{WTD}$$

where

$$ \dot{S}_{aff} = \frac{1}{\mathcal{T}} \sum_{\alpha \beta} p_{\beta \alpha} R_\alpha \ln{p_{\beta \alpha}/p_{\alpha \beta}} $$

and

$$\dot{S}_{WTD} = \frac{1}{\mathcal{T}} \sum_{\alpha \beta \mu} p_{\mu \beta} p_{\beta \alpha} R_\alpha \mathcal{D} [\Psi(t | \beta \to \mu ) || \Psi (t | \beta \to \alpha)]$$

In [19]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import rv_continuous

# Define the KL divergence between two waiting time distributions
def kullback_leibler_divergence(dist1, dist2):
    """Compute the Kullback-Leibler divergence between two distributions."""
    def integrand(x):
        p = dist1.pdf(x)
        q = dist2.pdf(x)
        return p * np.log(p / q) if p > 0 and q > 0 else 0
    
    return quad(integrand, 0, np.inf)[0]

# Compute the irreversibility signature from waiting time distributions
def compute_irreversibility(states, transition_matrix, waiting_time_distributions):
    """Compute the irreversibility measure from waiting time distributions."""
    stationary_distribution = compute_stationary_distribution(transition_matrix)
    S_WTD = 0

    for i, state_i in enumerate(states):
        for j, state_j in enumerate(states):
            if i != j and transition_matrix[i, j] > 0 and transition_matrix[j, i] > 0:
                P_ij = transition_matrix[i, j] * stationary_distribution[i]
                waiting_time_ij = waiting_time_distributions.get((state_i, state_j))
                waiting_time_ji = waiting_time_distributions.get((state_j, state_i))
                
                if waiting_time_ij and waiting_time_ji:
                    S_WTD += P_ij * kullback_leibler_divergence(waiting_time_ij, waiting_time_ji)
    
    return S_WTD

def compute_stationary_distribution(transition_matrix):
    """Compute the stationary distribution of a Markov chain."""
    eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
    stationary = eigvecs[:, np.isclose(eigvals, 1)]
    stationary = stationary / stationary.sum()
    return stationary.real.flatten()

# Define a custom waiting time distribution (Exponential)
class CustomExponential(rv_continuous):
    """Custom exponential waiting time distribution."""
    def __init__(self, rate):
        super().__init__()
        self.rate = rate

    def _pdf(self, x):
        return self.rate * np.exp(-self.rate * x) if x >= 0 else 0

    def rvs(self, size=1):
        return np.random.exponential(1 / self.rate, size=size)

Irreversibility Signature from Waiting Time Distributions: 0.039863895456331976


In [29]:
# Example Usage
states = ["A", "B", "C"]
transition_matrix = np.array([
    [0.0, 0.6, 0.4],
    [0.5, 0.0, 0.5],
    [0.3, 0.7, 0.0],
])

# Sample rates from a log-normal distribution
np.random.seed(42)
sampled_rates = np.random.lognormal(mean=0, sigma=0.5, size=6)

# Define waiting time distributions
waiting_time_distributions = {
    ("A", "B"): CustomExponential(rate=sampled_rates[0]),
    ("B", "A"): CustomExponential(rate=sampled_rates[1]),
    ("A", "C"): CustomExponential(rate=sampled_rates[2]),
    ("C", "A"): CustomExponential(rate=sampled_rates[3]),
    ("B", "C"): CustomExponential(rate=sampled_rates[4]),
    ("C", "B"): CustomExponential(rate=sampled_rates[5]),
}

# Compute irreversibility measure
S_WTD = compute_irreversibility(states, transition_matrix, waiting_time_distributions)
print("Irreversibility Signature from Waiting Time Distributions:", S_WTD)

Irreversibility Signature from Waiting Time Distributions: 0.039863895456331976


# Sampling parameters for the 3-vertex graph $K$

Parameters are defined as $10^x$, where $x$ is uniformly sampled from $(-3, 3)$. The user can specify the endpoints of this range to sample parameter space more broadly, but $-3$ and $3$ are set as the default parameters.

In [20]:
params = random_parameters()

In [21]:
print(params)

[2.23101080e-03 1.57418900e+02 4.04287274e+00 1.77188474e+01
 1.32894487e-03 6.59871107e+02]


The parameters can be determined such that they satisfy the cycle condition on $K$, initializing the graph in an equlibrium steady state.

In [38]:
params = equilibrium_parameters()
params.dtype

dtype('float128')

In [39]:
print(params)

[6.59871107e+02 9.87770029e+01 1.87946682e-02 1.23296232e-02
 1.26016397e-02 1.23748211e-03]


To generate the figure in the paper, we will manually assign values for the edge labels of the 3-vertex graph. The parameters for the 3-vertex graph are listed in the following order: $[a, b, d, c, f, e] = [\ell(1\to 2), \ell(2\to 1), \ell(2\to 3), \ell(3\to 2), \ell(1\to 3), \ell(3\to 1)] = [\ell(A\to B), \ell(B\to A), \ell(B\to C), \ell(C\to B), \ell(A\to C), \ell(C\to A)]$

In [40]:
#params = [0.08833331, 0.44953403, 0.58597936, 0.02356496, 0.00747019, 0.75898155]

Next, we compute the Laplacian matrix $\mathcal{L}(K)$ for this specific parameterzation of $K$, and the steady state distribution $\pi(K)$ as computed through the Matrix-Tree Theorem.

In [47]:
L = Laplacian_K(params)
print(L)

[[-6.59883709e+02  9.87770029e+01  1.23748211e-03]
 [ 6.59871107e+02 -9.87957976e+01  1.23296232e-02]
 [ 1.26016397e-02  1.87946682e-02 -1.35671053e-02]]


dtype('float128')

In [42]:
pi = steady_state_MTT_K(params)
print(pi)

[0.05597944 0.37396571 0.57005485]


We can also calculate the cycle affinity $\tilde{A}(C)$ for $K$. The cycle affinity  quantifies the departure from equilibrium that arises from the cycle $C$. We take the absolute value of the cycle affinity in our calculation.

In [43]:
cycle_affinity_K(params)

0.0

In [44]:
params.dtype

dtype('float128')

In [50]:
# Compute irreversibility measure
states = ["A", "B", "C"]

new_params = params.astype(dtype=float)
print(new_params.dtype)

new_L = L.astype(dtype=float)

waiting_time_distributions = {
    ("A", "B"): CustomExponential(rate=new_params[0]),
    ("B", "A"): CustomExponential(rate=new_params[1]),
    ("B", "C"): CustomExponential(rate=new_params[2]),
    ("C", "B"): CustomExponential(rate=new_params[3]),
    ("A", "C"): CustomExponential(rate=new_params[4]),
    ("C", "A"): CustomExponential(rate=new_params[5]),
}

S_WTD = compute_irreversibility(states, new_L, waiting_time_distributions)
print("Irreversibility Signature from Waiting Time Distributions:", S_WTD)

float64


IndexError: index 0 is out of bounds for axis 0 with size 0